In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle



# Load preprocessed data
train_input = np.load('/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_input.npy')
train_output = np.load('/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/train_output.npy')
test_input = np.load('/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/test_input.npy')
test_output = np.load('/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/test_output.npy')
val_input = np.load('/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/val_input.npy')
val_output = np.load('/Volumes/Personal/SDP/Brain tumor MRI and CT scan/data(processed)/val_output.npy')

# Shuffle the training data
train_input, train_output = shuffle(train_input, train_output, random_state=42)

# Preprocessing functions
def normalize_image(img):
    # Normalization function
    return cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

def preprocess_data(input_data, output_data):
    # Normalize input and output data
    input_data_norm = np.array([normalize_image(img) for img in input_data])
    output_data_norm = np.array([normalize_image(img) for img in output_data])
    return input_data_norm, output_data_norm

# Preprocess the data
train_input, train_output = preprocess_data(train_input, train_output)
val_input, val_output = preprocess_data(val_input, val_output)

# Split the training data into train and validation sets
train_input, test_input, train_output, test_output = train_test_split(train_input, train_output, test_size=0.2, random_state=42)

print("Data preprocessing completed.")


Data preprocessing completed.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

# Define U-Net model
def unet_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Contracting path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Expansive path
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv5))
    merge6 = Concatenate()([conv4, up6])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define the model
model1 = unet_model(input_shape=(None, None, 1))

# Compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model1.fit(train_input, train_output, validation_data=(val_input, val_output), epochs=10, batch_size=16, verbose=1)

print("Model-1 training completed.")


/Users/chintubharath/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 328s 11s/step - accuracy: 0.3193 - loss: 0.6280 - val_accuracy: 0.3515 - val_loss: 0.4841
Epoch 2/10
 7/29 ━━━━━━━━━━━━━━━━━━━━ 3:58 11s/step - accuracy: 0.3498 - loss: 0.4454